In [62]:
import pyspark
import os
import sys
from pyspark import SparkContext
from pyspark import SparkConf
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [63]:
#Q1, Q2 and Q4

from pyspark.sql import SparkSession
# Create a SparkSession
spark = SparkSession.builder.getOrCreate()
# Create a DataFrame
df = spark.createDataFrame([
(1, "Alice"),
(2, "Bob"),
(3, "Carol"),
(4, "Dave"),
(5, "Eve")
], ["id", "name"])
print(df.count())
df.show()
#df.write.format("csv").mode('overwrite').save('output')
# Filter the DataFrame to only include rows where the name starts with "A"
df = df.filter(df["name"].startswith("A"))
# Add a new column to the DataFrame called "age"
df = df.withColumn("age", df["id"] * 10)

# Print the DataFrame
df.show()

/home/lplab/anaconda3/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


5
+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
|  3|Carol|
|  4| Dave|
|  5|  Eve|
+---+-----+

+---+-----+---+
| id| name|age|
+---+-----+---+
|  1|Alice| 10|
+---+-----+---+



In [64]:
#Q3

import pandas as pd
from pyspark.sql import functions as F
df_pd = pd.DataFrame(
data={'integers': [1, 2, 3],
'floats': [-1.0, 0.5, 2.7],
'integer_arrays': [[1, 2], [3, 4, 5], [6, 7, 8, 9]]}
)
df = spark.createDataFrame(df_pd)
df.printSchema() # It will print the Schema
from pyspark.sql.functions import col, avg, sum, min, max

df.agg(sum("integers")).show()
df.agg(avg("integers")).show()
df.select(avg(col("integers"))).show()
df.agg(min("integers")).show()
df.agg(max("integers")).show()

df.show()

root
 |-- integers: long (nullable = true)
 |-- floats: double (nullable = true)
 |-- integer_arrays: array (nullable = true)
 |    |-- element: long (containsNull = true)

+-------------+
|sum(integers)|
+-------------+
|            6|
+-------------+

+-------------+
|avg(integers)|
+-------------+
|          2.0|
+-------------+

+-------------+
|avg(integers)|
+-------------+
|          2.0|
+-------------+

+-------------+
|min(integers)|
+-------------+
|            1|
+-------------+

+-------------+
|max(integers)|
+-------------+
|            3|
+-------------+

+--------+------+--------------+
|integers|floats|integer_arrays|
+--------+------+--------------+
|       1|  -1.0|        [1, 2]|
|       2|   0.5|     [3, 4, 5]|
|       3|   2.7|  [6, 7, 8, 9]|
+--------+------+--------------+



In [65]:
#Q5

# Create a SparkSession
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
# Load the text file
lines = sc.textFile("sentence.txt")
counts = lines.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y)
output = counts.collect()
for (word, count) in output:
    print("%s: %i" % (word, count))
# Split the lines into words

name: 1
is: 1
My: 1
Prahlad: 1
and: 1
Iam: 1
the: 1
Greatest: 1


In [66]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
# Create a SparkSession
spark = SparkSession.builder.getOrCreate()
# Load the text file
lines = spark.read.text("sentence.txt")
# Split the lines into words
words = lines.withColumn('word', f.explode(f.split(f.col('value'), ' '))).groupBy('word').count().sort('count', ascending=False).show()
# Stop the SparkSession
spark.stop()

+--------+-----+
|    word|count|
+--------+-----+
|    name|    1|
| Prahlad|    1|
|      is|    1|
|     the|    1|
|     and|    1|
|Greatest|    1|
|     Iam|    1|
|      My|    1|
+--------+-----+

